In [ ]:
import os

import numpy
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import mode

from hera_cal.redcal import get_reds

from red_likelihood import fltBad, groupBls, group_data
from red_utils import find_flag_file, find_nearest, find_zen_file, get_bad_ants, \
match_lst, split_rel_results

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

In [ ]:
JD = 2458098.43869
pol = 'ee'
freq_channel = 620

In [ ]:
zen_fn = find_zen_file(JD)
bad_ants = get_bad_ants(zen_fn)
flags_fn = find_flag_file(JD, 'first')

In [ ]:
hdraw, RedG, cMData = group_data(zen_fn, pol, freq_channel, None, bad_ants, flags_fn)
cData_1 = numpy.squeeze(cMData.filled())

##  Visibilities for a given baseline type

In [ ]:
reds = fltBad(get_reds(hdraw.antpos, pols=[pol]), bad_ants)
bl_types = RedG[:, 0]
slct_bl_type = mode(bl_types)[0][0] # selecting modal value for baseline type
slct_idxs = numpy.where(bl_types == slct_bl_type)[0]
print('Selecting visibilities with baseline type {} that are redundant with '\
      'baseline {}, of which there are {}'.format(slct_bl_type, reds[slct_bl_type][0], \
      slct_idxs.size))

In [ ]:
slct_vis_t = cData_1[:, slct_idxs]

In [ ]:
vis_amp_t = numpy.abs(slct_vis_t)

fig, ax = plt.subplots(figsize=(12, 8))
plt.plot(vis_amp_t, alpha=0.5)
plt.plot(numpy.median(vis_amp_t, axis=1), linewidth=3)
plt.xlabel('Time integration')
plt.ylabel('Visibility amplitude')
plt.show()

In [ ]:
df_t = pd.DataFrame(vis_amp_t).stack().reset_index()
df_t.rename(columns={'level_0': 'time_int', 'level_1': 'bl', 0: 'vis_amp'}, inplace=True)

fig, ax = plt.subplots(figsize=(12, 8))
ax = sns.lineplot(x='time_int', y='vis_amp', data=df_t, ci='sd', linewidth=3)
plt.xlabel('Time integration')
plt.ylabel('Visibility amplitude')
plt.show()

In [ ]:
vis_phase_t = numpy.angle(slct_vis_t)

fig, ax = plt.subplots(figsize=(12, 8))
plt.plot(vis_phase_t)
plt.xlabel('Time integration')
plt.ylabel('Visibility phase')
plt.show()

## Upper bound on noise

### Adjacent time integrations

Take a given baseline, and compare the visibilities between one time integration and the next

In [ ]:
vis_bl_t = slct_vis_t[:, 0]
vis_diff_t = numpy.asarray([t - s for s, t in zip(vis_bl_t, vis_bl_t[1:])])

Take the standard deviation of the different between visibilities adjacent in time for an entire dataset, as a proxy for the noise

n.b. the variance of a complex random variable $z$ is equal to the sum of the variances of its real and imaginary parts

$$ \mathrm{Var}[z] = \mathrm{Var}[\Re(z)] + \mathrm{Var}[\Im(z)] $$

In [ ]:
noise_std_t = numpy.sqrt(numpy.var(vis_diff_t.real) + numpy.var(vis_diff_t.imag))
mean_vis_amp_t = numpy.mean(numpy.abs(vis_bl_t))
print('Upper bound on noise, by comparing adjacent visibilities in time at the same '\
      'frequency is {}, which is {}% of the mean visibility amplitude'.\
      format(round(noise_std_t, 5), round(100*noise_std_t/mean_vis_amp_t, 1)))

### Same LAST on different JDs

A more reliable estimate of the noise is by looking at visibilities from different JDs that match in LAST

In [ ]:
JD_2a= match_lst(JD, 2458099) # finding the JD_time of the zen_file
# that matches the LAST of the first dataset used
zen_fn2a = find_zen_file(JD_2a)
bad_ants2a = get_bad_ants(zen_fn2a)
flags_fn2a = find_flag_file(JD_2a, 'first')

In [ ]:
last_df = pd.read_pickle('jd_lst_map_idr2.pkl')

next_row = numpy.where(last_df['JD_time'] == JD_2a)[0][0] + 1
JD_2b = last_df.iloc[next_row]['JD_time']
zen_fn2b = find_zen_file(JD_2b)
bad_ants2b = get_bad_ants(zen_fn2b)
flags_fn2b = find_flag_file(JD_2b, 'first')

In [ ]:
last1 = last_df[last_df['JD_time'] == JD]['LASTs'].values[0]
last2 = last_df[last_df['JD_time'] == JD_2a]['LASTs'].values[0]
_, offset = find_nearest(last2, last1[0])

In [ ]:
_, _, cMData = group_data(zen_fn2a, pol, freq_channel, None, bad_ants2a, flags_fn2a)
cData_2a = numpy.squeeze(cMData.filled())[offset:]

_, _, cMData = group_data(zen_fn2b, pol, freq_channel, None, bad_ants2b, flags_fn2b)
cData_2b = numpy.squeeze(cMData.filled())[:offset]


In [ ]:
cData_2 = numpy.concatenate((cData_2a, cData_2b))
slct_vis_j = cData_2[:, slct_idxs]

In [ ]:
vis_amp_j = numpy.abs(slct_vis_j)

fig, ax = plt.subplots(figsize=(12, 8))
plt.plot(vis_amp_j, alpha=0.5)
plt.plot(numpy.median(vis_amp_j, axis=1), linewidth=3)
plt.xlabel('Time integration')
plt.ylabel('Visibility amplitude')
plt.show()

In [ ]:
# Plotting median visibilities on separate JD days but same LAST on top of each other

fig, ax = plt.subplots(figsize=(12, 8))
plt.plot(numpy.median(vis_amp_t, axis=1), linewidth=2)
plt.plot(numpy.median(vis_amp_j, axis=1), linewidth=2)
plt.xlabel('Time integration')
plt.ylabel('Visibility amplitude')
plt.show()

In [ ]:
vis_bl_j = slct_vis_j[:, 0]
vis_diff_j = numpy.asarray([t - s for s, t in zip(vis_bl_j, vis_bl_j[1:])])

In [ ]:
noise_std_j = numpy.sqrt(numpy.var(vis_diff_j.real) + numpy.var(vis_diff_j.imag))
mean_vis_amp_j = numpy.mean(numpy.abs(vis_bl_j))
print('Upper bound on noise, by comparing adjacent visibilities in time at the same '\
      'frequency is {}, which is {}% of the mean visibility amplitude'.\
      format(round(noise_std_j, 5), round(100*noise_std_j/mean_vis_amp_j, 1)))